# Dog breed prediction for Google Cloud ML

In [ ]:
# !pip install cloudmlmagic   # Python 2 only
# !pip install --upgrade google-cloud-vision   # Face detector

In [1]:
%load_ext cloudmlmagic2  # My own version

In [2]:
%ml_init -projectId udacity-190420 -bucket udacity-190420/dog-breed -requires keras -scaleTier BASIC -region europe-west1 -runtimeVersion 1.2

{'install_requires': ['keras', 'h5py', 'Pillow']}

In [ ]:
%%ml_code

from keras.applications.inception_v3 import InceptionV3 as keras_model

selected_model= keras_model(weights='imagenet', include_top=False)

In [ ]:
from keras.models import load_model
my_model = load_model('../saved_models/weights.best.{}.hdf5'.format(NETWORK))

In [238]:
#%%ml_code
NETWORK= 'InceptionV3'

from keras.applications.inception_v3 import InceptionV3 as keras_model

from keras.models import load_model
from keras import backend as K
import tensorflow as tf

def model2pb(model, file_name):
    num_output= 1
    output_node_prefix= 'output_node'
    K.set_learning_phase(0)
    pred = [None]*num_output
    pred_node_names = [None]*num_output
    for i in range(num_output):
        pred_node_names[i] = output_node_prefix+str(i)
        pred[i] = tf.identity(model.outputs[i], name=pred_node_names[i])
    print('output nodes names are: ', pred_node_names)
    sess = K.get_session()
    K.get_session().graph
    
    from tensorflow.python.framework import graph_util
    from tensorflow.python.framework import graph_io
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    graph_io.write_graph(constant_graph, '.', file_name, as_text=False)

selected_model= keras_model(weights='imagenet', include_top=False)
my_model = load_model('../saved_models/weights.best.{}.hdf5'.format(NETWORK))
# Concatenate both models
merged_model = Model(inputs= [selected_model.input], outputs= [my_model(selected_model.output)])
#merged_model.save('final_model.h5')
#merged_model= load_model('final_model.h5')
#model2pb(merged_model, 'final_model.pb')

Test

In [239]:
import numpy as np
from keras.applications.inception_v3 import preprocess_input as keras_model_preprocess_input
from keras.preprocessing import image 

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def predict(img_path):
    prediction= merged_model.predict(keras_model_preprocess_input(path_to_tensor(img_path)))
    max= np.argmax(prediction)
    return prediction[0, max]* 100

In [240]:
from glob import glob

files= sorted(glob("../test/*"))
for img_path in files:
    acc= predict(img_path)
    print('%30s : %.2f' % (img_path, acc))

          ../test/amparito.jpg : 99.94
         ../test/angelina.jpeg : 33.12
             ../test/aznar.jpg : 44.22
            ../test/begle.jpeg : 100.00
              ../test/cat.jpeg : 61.08
        ../test/chihuahua.jpeg : 100.00
         ../test/doberman.jpeg : 100.00
             ../test/dog1.jpeg : 93.66
             ../test/dog2.jpeg : 54.97
              ../test/duki.jpg : 99.99
      ../test/jack_sparrow.jpg : 69.99
             ../test/male1.jpg : 41.44
     ../test/maxresdefault.jpg : 62.94
              ../test/rosa.jpg : 40.99
             ../test/trump.jpg : 60.19
